In [2]:
import argparse
import pathlib
import os
import json
import subprocess
import uuid
from funcx.sdk.client import FuncXClient
from globus_sdk import TransferClient
import globus_sdk

In [187]:
DEBUG = False

In [167]:
def hello_world():
    return "Hello World!"

# Start up a funcx client -- only do this once!
from funcx.sdk.client import FuncXClient
fxc = FuncXClient()
func_uuid = fxc.register_function(hello_world) # function uuid to be registered
tutorial_endpoint = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint

In [111]:
tutorial_endpoint_1 = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"      # endpoint "Globus Tutorial Endpoint 1"
tutorial_endpoint_2 = "ddb59af0-6d04-11e5-ba46-22000b92c6ec"      # endpoint "Globus Tutorial Endpoint 2"


# As in the Platform_Introduction_Native_App_Auth notebook, do the Native App Grant Flow
CLIENT_ID = "3b1925c0-a87b-452b-a492-2c9921d3bd14"
SCOPES = "openid profile email urn:globus:auth:scope:transfer.api.globus.org:all urn:globus:auth:scope:auth.globus.org:view_identities"
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow(requested_scopes=SCOPES)
print(f"Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}")

Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=3b1925c0-a87b-452b-a492-2c9921d3bd14&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall+urn%3Aglobus%3Aauth%3Ascope%3Aauth.globus.org%3Aview_identities&state=_default&response_type=code&code_challenge=GpxpSB9OuGXYQiAOfXaNYfYbaVgDaULUTA25DX-5Uws&code_challenge_method=S256&access_type=online


In [182]:
# Fill this line in with the code that you got
auth_code = "gfjIP3eCsDSWEdljciw9yCrxkd8XI4"

token_response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)
transfer_access_token = token_response.by_resource_server['transfer.api.globus.org']['access_token']
transfer_authorizer = globus_sdk.AccessTokenAuthorizer(transfer_access_token)

# Your TransferClient is named "tc"
tc = globus_sdk.TransferClient(authorizer=transfer_authorizer)
print(f"TransferClient:\n\n{tc}")

AuthAPIError: (400, 'Error', 'invalid_grant')

In [184]:
class UserOptions:
  def configure_ep(self):
    if tc is None:
        print ("Cannot create Xtract Endpoint -- no globus transfer client provided")
        return False
    if self.configure is None:
      print ("Cannot create Xtract Endpoint -- no endpoint name provided.")
      return False
    
    # Load user options stored locally
    if os.path.isfile("/Users/joaovictor/" + self.configure +  "/config.json"):
      print ("Found config file in /Users/joaovictor/" + self.configure +  "/config.json") 
      data = None
      with open("/Users/joaovictor/" + self.configure +  "/config.json") as f:
        data = json.loads(f.read())
        for key in data.keys():
          if DEBUG: print (key, data[key])
          if hasattr (self, key):
            setattr (self, key, data[key])
    else:
      print ("No config file found or provided.")
    
    if not self.globus_eid:
      print("Cannot create Xtract Endpoint -- missing globus_eid")
      return False
    if not self.local_metadata:
      print("Cannot create Xtract Endpoint -- missing local metadata")
      return False
    if not self.metadata_write_dir:
      print("Cannot create Xtract Endpoint -- missing metadata write directory")
      return False
    print ("Created Xtract Endpoint!")
    return True

  def __init__(self):
    self.tc = None
    self.configure = None
    self.globus_eid = None
    self.funcx_eid = None
    self.local_metadata = None
    self.metadata_write_dir = None

In [190]:
'''
Checks whether a funcx endpoint and globus endpoint are online, returning the result as a dictionary.
Might be good to perform some basic error checking, particularly around funcx_response
'''
def is_online(ops:UserOptions):
    res = {}
    funcx_response = fxc.run(endpoint_id=ops.funcx_eid, function_id=func_uuid)
    globus_response = None
    
    if ops.funcx_eid is None:
        res['funcx_eid'] = 'Funcx not present.'
    else:
        if fxc.get_result(funcx_response) == 'Hello World!':
            res['funcx_eid'] = 'OK'
        else:
            res['funcx_eid'] = f'Not OK -- Expected: \'Hello World!\', but received: \'{fxc.get_result(funcx_response)}\''
    
    if ops.globus_eid is None:
        res['globus_eid'] = 'Globus not present.'
    else:
        endpoint = tc.get_endpoint(ops.globus_eid)
        if endpoint is not None and endpoint["DATA"][0]["is_connected"]:
            res['globus_eid'] = 'OK'
        else:
            res['globus'] = 'Not OK'
    return res

In [191]:
ops = UserOptions()
ops.tc = tc
ops.configure = 'test'
ops.configure_ep()

print(is_online(ops))

Found config file in /Users/joaovictor/test/config.json
Created Xtract Endpoint!
{'funcx_eid': 'OK', 'globus_eid': 'OK'}
